### Homework
Ваша задача спарсить информацию о компаниях, находящихся в индексе S&P 500 с данного сайта: <br>
https://markets.businessinsider.com/index/components/s&p_500

Для каждой компании собрать следующую информацию:
* Текущая стоимость в рублях (конвертацию производить по текущему курсу, взятому с сайта [центробанка РФ](http://www.cbr.ru/development/sxml/))
* Код компании (справа от названия компании на странице компании)
* P/E компании (информация находится справа от графика на странице компании)
* Годовой рост/падение компании в процентах (основная таблица)
* Высчитать какую прибыль принесли бы акции компании (в процентах), если бы они были куплены на уровне 52 Week Low и проданы на уровне 52 Week High (справа от графика на странице компании)

Сохранить итоговую информацию в 4 JSON файла:
1. Топ 10 компаний с самими дорогими акциями в рублях.
2. Топ 10 компаний с самым низким показателем P/E.
3. Топ 10 компаний, которые показали самый высокий рост за последний год
4. Топ 10 комппаний, которые принесли бы наибольшую прибыль, если бы были куплены на самом минимуме и проданы на самом максимуме за последний год.
<br>Пример формата:
```
[
{
    "code": "MMM",
    "name": "3M CO.",
    "price" | "P/E" | "growth" | "potential profit" : value,
},
...
]
```
For scrapping you cans use `beautifulsoup4` <br>
For requesting `aiohttp`

In [62]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [19]:
#получаем ссылки на все компании в списке и, заодно, годовой рост/падение компании
company_links = []
growth = []
names = []
for i in range(1,11):
    url = ('https://markets.businessinsider.com/index/components/s&p_500?p='+str(i))
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    
    for company_href in bs.find_all(name = "td", attrs = {"class": "table__td table__td--big"}):
        name = company_href.find("a").get_text()
        names.append(name)
        link = company_href.find(name = "a", href = True).get('href')
        company_links.append('https://markets.businessinsider.com'+link)
    
    tables = bs.find_all(name = "tr")
    for table in tables[1:]:
        columns = table.find_all(name = "td")
        growth.append(float(columns[7].text.split()[-1][:-1]))

In [58]:
#Ищем стоимость компании, код и P/E
prices = []
codes = []
pe_ratios = []

for url in company_links: 
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    
    price = bs.find(name = "span", attrs = {"class": "price-section__current-value"}).text
    prices.append(float(price.replace(',','')))

    code = bs.find(name = "span", attrs = {"class": "price-section__category"}).text
    codes.append(code[-4:].rstrip())

    right_data_items= bs.find_all(name = "div", attrs = {"class": "snapshot__data-item padding-right--zero"})
    for data_item in right_data_items:
        if data_item.find(name ="div", attrs = {"class": "snapshot__header"}, string ="P/E Ratio"):
            try:
                pe_ratios.append(float(data_item.text.replace("P/E Ratio","").replace(",","")))
            except ValueError:
                pe_ratios.append(0)


In [26]:
#Ищем и высчитываем разницу 52 Week Low и 52 Week High
profits = []

for url in company_links: 
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    try:
        small_data_items = bs.find_all(name = "div", attrs = {"class": "snapshot__data-item snapshot__data-item--small"})
        if len(small_data_items) == 0:
            ft_week_low = 0
        else:
            ft_week_low = float(small_data_items[len(small_data_items)-1].text.replace("52 Week Low","").replace(',',''))
        
        small_data_items_right = bs.find_all(name = "div", attrs = {"class": "snapshot__data-item snapshot__data-item--small snapshot__data-item--right"})
        if len(small_data_items_right) == 0:
            ft_week_high = 0
        else:
            ft_week_high = float(small_data_items_right[len(small_data_items_right)-1].text.replace("52 Week High","").replace(',',''))
        profits_persentage = round(((ft_week_high - ft_week_low)/ft_week_low)*100, 2)
    except ValueError:
        profits_persentage = 0
    
    profits.append(profits_persentage)

In [73]:
#получаем курс доллара
data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
usd_cur = float(data['Valute']['USD']['Value'])

In [83]:
prices = [pr * usd_cur for pr in prices]

In [86]:
#Записываем все в один датасет
sp_500 = {
    "code": codes,
    "name": names,
    "price": prices,
    "P/E": pe_ratios,
    "growth":growth,
    "potential profit": profits}
sp_500 = pd.DataFrame(sp_500)

In [87]:
sp_500

,code,name,price,P/E,growth,potential profit
0,MMM,3M,11346.864672,17.09,47.86,72.44
1,AOS,A.O. Smith,5375.048147,18.80,-26.66,57.04
2,ABT,Abbott Laboratories,10651.736963,14.81,21.47,44.19
3,BBV,AbbVie,15382.076890,74.34,5.51,42.36
4,ACN,Accenture,24259.213150,29.90,-8.09,44.32
...,...,...,...,...,...,...
447,WTW,Willis Towers Watson,24974.178035,29.00,23.14,39.35
448,YNN,Wynn Resorts,6788.446937,95.62,-18.49,65.83
449,XEL,Xcel Energy,5703.188100,66.66,29.32,41.18
450,XYL,Xylem,9597.060439,67.42,-13.25,45.14


In [94]:
#Переводим в json
sp_500.sort_values(by='price',ascending=False).iloc[:10].to_json(indent=4, path_or_buf='top10_highest_prices.json')
sp_500.sort_values(by='P/E',ascending=True).iloc[:10].to_json(indent=4, path_or_buf='top10_lowest_pe.json')
sp_500.sort_values(by='growth',ascending=False).iloc[:10].to_json(indent=4, path_or_buf='top10_highest_growth.json')
sp_500.sort_values(by='potential profit',ascending=False).iloc[:10].to_json(indent=4,path_or_buf='top10_highest_potential_profit.json')